# Pipeline for calculating overbite

## Initialization
We assume that the images used for our pipeline is transformed and aligned to the standard of the images used in training. This can be done with a preprocessing function.

In [5]:
# Importing packages
import os
import torch
import re
import torchvision.transforms as T
from collections import defaultdict
from PIL import Image

# Opdeling af billeder i Lower/Upper og Left/Right/Middle 

In [14]:
%run Opdeling_og_flip_af_billeder.py

Loaded 2 images from Pipeline_data/Raw_data: ['0DNK2I7H_lower_combined.png', '0DNK2I7H_upper_combined.png']
✅ All images have dimensions: 3072x1024
✅ Processed 2 images and saved to Pipeline_data/Clean Data


# Indlæs den trænede model og kør modellen

In [ ]:
%run Run_model.py

# Pixelmatrix på data 

In [22]:
%run Pixelmatrix.py 

  0%|          | 0/400 [00:00<?, ?it/s]

Missing: GNR6QR3P_lower_right.png
Missing: 5LP835GA_upper_right.png
Missing: 0191WKPV_lower_right.png
Missing: 01E5XG8Z_lower_left.png
Missing: 01ETW4JZ_upper_right.png


  2%|▏         | 6/400 [00:00<00:52,  7.56it/s]

Missing: 01FURZ62_lower_right.png
Missing: EB6DQGAR_lower_right.png
Missing: 0145NKWZ_lower_left.png
Missing: MNIAB8K3_lower_left.png
Missing: 0158T9ED_upper_left.png
Missing: 013NXP1H_upper_left.png
Missing: 9GSQBATD_lower_right.png
Missing: 0AAQ6BO3_upper_right.png
Missing: HZH8DYC7_lower_right.png
Missing: 015WW8D6_lower_right.png
Missing: 0169NHT6_upper_left.png
Missing: ANLLPLV7_upper_left.png
Missing: 0145NKWZ_upper_right.png
Missing: KBGVMAF7_upper_left.png
Missing: 01HMZ9RN_lower_right.png
Missing: 01ETW4JZ_lower_left.png
Missing: 013ZD6EX_upper_right.png
Missing: 01FAYE3T_upper_right.png
Missing: 0158T9ED_lower_right.png
Missing: BC0OWLNW_lower_left.png
Missing: 015WW8D6_upper_left.png
Missing: 01G1AKV2_lower_right.png
Missing: 015WXFRN_upper_right.png
Missing: 0158T9ED_upper_right.png
Missing: IHKL3Y8N_lower_left.png
Missing: 4J24X0ES_lower_right.png


  8%|▊         | 33/400 [00:01<00:14, 25.74it/s]

Missing: 01E5XG8Z_upper_right.png
Missing: 013NXP1H_upper_right.png
Missing: 0KIUZ4G5_lower_left.png
Missing: LGADV15G_lower_right.png
Missing: CBSF6Q84_lower_right.png
Missing: 01ETW4JZ_upper_left.png
Missing: 0148UTKX_upper_left.png


 10%|█         | 41/400 [00:02<00:17, 20.92it/s]

Missing: 013NXP1H_lower_left.png
Missing: E552BK9X_upper_right.png
Missing: 0VFZZY32_lower_left.png
Missing: 014ZTUSK_lower_left.png
Missing: 01KRCSKD_upper_left.png
Missing: JJEIMAPR_lower_right.png
Missing: IHKL3Y8N_upper_left.png
Missing: 56R9FBMK_lower_right.png
Missing: 015WXFRN_upper_left.png
Missing: 017UYAW4_upper_right.png
Missing: 01J9RWK6_upper_right.png
Missing: 013NUWYR_lower_right.png
Missing: 01J1XXHR_lower_right.png
Missing: 016XWJ8X_upper_right.png
Missing: 01JH0154_upper_left.png
Missing: BC0OWLNW_lower_right.png
Missing: 0KIUZ4G5_lower_right.png
Missing: 9GSQBATD_upper_left.png
Missing: 015KFJV6_upper_right.png
Missing: 01J1XXHR_upper_left.png
Missing: KWWQJ60M_lower_left.png
Missing: B7JSFJPG_upper_left.png
Missing: 5SD5BWLR_upper_left.png
Missing: LA67YYVJ_lower_right.png
Missing: 01FURZ62_lower_left.png
Missing: KBGVMAF7_lower_left.png
Missing: 014EZU5V_upper_right.png
Missing: J1A5FRQN_lower_left.png
Missing: 9GSQBATD_lower_left.png
Missing: 014JJ942_upper_left.p

100%|██████████| 400/400 [00:02<00:00, 147.98it/s]

Missing: J1A5FRQN_lower_right.png
Missing: LGADV15G_upper_right.png
Missing: 0OF8OOCX_upper_right.png
Missing: K9X2TVDE_upper_left.png
Missing: 3JCUF41E_upper_left.png
Missing: ANLLPLV7_lower_right.png
Missing: 014WKP6A_upper_right.png
Missing: 015KFJV6_upper_left.png
Missing: 01HJKFR8_upper_right.png
Missing: 0145NKWZ_lower_right.png
Missing: 014EZU5V_lower_right.png
Missing: 01HJKFR8_lower_right.png
Missing: KS0FGXGB_upper_right.png
Missing: 7IGA0W6E_upper_left.png
Missing: B7JSFJPG_lower_left.png
Missing: 87N5YSES_upper_right.png
Missing: 7DRMXZ3A_upper_left.png
Missing: IHKL3Y8N_lower_right.png
Missing: 0TMOBYXS_upper_right.png
Missing: 0169NHT6_lower_left.png
Missing: 6TZ254SP_upper_right.png
Missing: 0OF8OOCX_upper_left.png
Missing: KBGVMAF7_upper_right.png
Missing: AEOQ4Q9Q_lower_right.png
Missing: 01CCUKV4_upper_left.png
Missing: 013TXGFK_upper_left.png
Missing: 01MAVHYZ_lower_left.png
Missing: 015DXGZH_upper_left.png
Missing: 01JZF0KX_upper_left.png
Missing: 56R9FBMK_upper_lef